<a href="https://colab.research.google.com/github/tejashreemore/Petrol-Price-Forecasting/blob/main/ARIMA_%26_AutoKERAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Using Auto ML**

## Installing Auto Keras

In [30]:
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc1


  Cloning https://github.com/keras-team/keras-tuner.git (to revision 1.0.2rc1) to /tmp/pip-req-build-dra6_8vq
  Running command git clone --filter=blob:none --quiet https://github.com/keras-team/keras-tuner.git /tmp/pip-req-build-dra6_8vq
  Running command git checkout -q 0fb69434a132093518e0e53d40020145ae192629
  Resolved https://github.com/keras-team/keras-tuner.git to commit 0fb69434a132093518e0e53d40020145ae192629
  Preparing metadata (setup.py) ... done
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2rc1-py3-none-any.whl size=85426 sha256=bb53e690d57f827c3c8020c09859252b2d8e6b67638a2df414af6e70fd1f5e55
  Stored in directory: /tmp/pip-ephem-wheel-cache-if0jekxn/wheels/b8/e6/44/4b6aa4a149abdabe9752582dbd7d7ae0848e87fe71cd2c59ed
Successfully built keras-tuner
  Attempting uninstall: keras-tuner
    Found existing installation: keras-tuner 1.4.6
    Uninstalling keras-tuner-1.4.6:
      Successfully uninstalled keras-tuner-1.4.6
ERROR: pip's dependency resolver does not cur

In [31]:
!pip install autokeras

  Using cached keras_tuner-1.4.6-py3-none-any.whl (128 kB)
  Attempting uninstall: keras-tuner
    Found existing installation: keras-tuner 1.0.2rc1
    Uninstalling keras-tuner-1.0.2rc1:
      Successfully uninstalled keras-tuner-1.0.2rc1


In [32]:
!pip show autokeras

Name: autokeras
Version: 1.1.0
Summary: AutoML for deep learning
Home-page: http://autokeras.com
Author: DATA Lab, Keras Team
Author-email: jhfjhfj1@gmail.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: keras-nlp, keras-tuner, packaging, pandas, tensorflow
Required-by: 


In [33]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# from math import sqrt
import tensorflow as tf
import autokeras as ak

In [34]:
data = pd.read_csv("/content/train_data.csv")
data.rename(columns={"Petrol (USD)": "Petrol"}, inplace = True)
data=data[data.Petrol<160].reset_index()['Petrol'] #remove outlier refer to data_analysis.ipynb

In [35]:
scaler=MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(np.array(data).reshape(-1,1))

In [36]:
training_size=int(len(df)*0.65)
test_size=len(df)-training_size
train_data,test_data=df[0:training_size,:],df[training_size:len(df),:1]

In [37]:
train_data.shape
test_data.shape

(284, 1)

**Data Preprocessing**

In [38]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)

In [39]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [40]:
X_train

array([[0.00559152, 0.00382578, 0.00309005, ..., 0.16627428, 0.16524426,
        0.1640671 ],
       [0.00382578, 0.00309005, 0.00206004, ..., 0.16524426, 0.1640671 ,
        0.16274279],
       [0.00309005, 0.00206004, 0.00103002, ..., 0.1640671 , 0.16274279,
        0.15538552],
       ...,
       [0.91024132, 0.91597999, 0.90847557, ..., 0.88419659, 0.88655091,
        0.88743378],
       [0.91597999, 0.90847557, 0.88802237, ..., 0.88655091, 0.88743378,
        0.87154208],
       [0.90847557, 0.88802237, 0.88449088, ..., 0.88743378, 0.87154208,
        0.87639788]])

In [41]:
y_train

array([0.16274279, 0.15538552, 0.14670394, 0.14420247, 0.15862272,
       0.16642142, 0.18422602, 0.19540906, 0.19894055, 0.21100647,
       0.21762802, 0.22130665, 0.22748676, 0.2405827 , 0.24367275,
       0.2532372 , 0.29664509, 0.30694526, 0.29723367, 0.28134197,
       0.28399058, 0.29487934, 0.29326074, 0.283402  , 0.26868746,
       0.24014126, 0.21188935, 0.19334903, 0.19334903, 0.19114185,
       0.19643908, 0.20114773, 0.20335491, 0.21115362, 0.21777516,
       0.22572101, 0.23057681, 0.22836963, 0.22719247, 0.22645674,
       0.2230724 , 0.22572101, 0.23513832, 0.24941142, 0.25853443,
       0.27310182, 0.29458505, 0.31106533, 0.32254267, 0.32739847,
       0.32636845, 0.32210124, 0.31121248, 0.30944673, 0.31312537,
       0.31238964, 0.30547381, 0.30988817, 0.31606828, 0.33357858,
       0.34682166, 0.34755739, 0.35079459, 0.34741024, 0.316804  ,
       0.29708652, 0.27472042, 0.25412007, 0.22910536, 0.20585639,
       0.18746321, 0.18231313, 0.17216009, 0.16965862, 0.16759

In [42]:
print(X_train.shape), print(y_train.shape)

(424, 100)
(424,)


(None, None)

In [43]:
reg = ak.StructuredDataRegressor(
    overwrite=True, max_trials=3
)

In [44]:
reg.fit(x=X_train, y=y_train, verbose=0)

In [45]:

# evaluate the model
mae, _  = reg.evaluate(X_test, ytest, verbose=0)
#print('MAE: %.3f' % mae)
# use the model to make a prediction
yhat_test = reg.predict(X_test)

# get the best performing model
model = reg.export_model()


6/6 [==============================] - 0s 4ms/step


In [46]:
# summarize the loaded model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 multi_category_encoding (M  (None, 100)               0         
 ultiCategoryEncoding)                                           
                                                                 
 dense (Dense)               (None, 128)               12928     
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 re_lu_1 (ReLU)              (None, 32)                0         
                                                             

In [51]:
yhat_train= reg.predict(X_train)

14/14 [==============================] - 0s 3ms/step


In [52]:
mean_squared_error(ytest,yhat_test)

0.005054416754428976

In [53]:
mean_squared_error(y_train,yhat_train)

0.0044248017773317215

# **Using ARIMA**

In [54]:
model1 = ARIMA(data.values, order=(5,1,0))
model_fit1 = model1.fit()
output1= model_fit1.forecast(steps=30)
output1

array([119.80670599, 119.52143354, 119.30634698, 119.14818605,
       119.02925494, 118.93899217, 118.87071727, 118.81917633,
       118.78030453, 118.75097725, 118.72884028, 118.71213018,
       118.69951765, 118.6899984 , 118.68281381, 118.67739118,
       118.6732984 , 118.67020932, 118.66787782, 118.66611809,
       118.66478993, 118.66378748, 118.66303088, 118.66245982,
       118.66202882, 118.66170351, 118.66145798, 118.66127267,
       118.6611328 , 118.66102723])